In [1]:
import Data.List.Split (splitOn)

main = do
    input <- readFile "data/input.txt"
    print $ filter allowQQQ $ lines input


allowQQQ :: String -> Bool
allowQQQ line = (getDomain line) == "qqq.com"
    where getDomain = last . splitOn "@" . last . splitOn " "

main

["1 aaaaa aa@qqq.com","2 bbbbb bb@qqq.com","3 ccccc cc@qqq.com"]

In [2]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE DeriveGeneric #-}

import Data.Aeson
import Control.Applicative
import qualified Data.ByteString.Lazy as B
import GHC.Generics

data Person = Person {   name :: String
                       , email :: String
                       , created_at :: String
                       , is_deleted :: Maybe Int
                       } deriving Generic

instance FromJSON Person

main :: IO ()
main = do
  input <- B.readFile "data/input.json"
  let mm = decode input :: Maybe Person
  case mm of
    Nothing -> print "error parsing JSON"
    Just m -> (putStrLn . printPerson) m

printPerson m = (show.name) m ++" has a email: "++ (show.email) m

main

"Lois Williams" has a email: "lois.williams50@example.com"

In [3]:
import Text.CSV

main :: IO ()
main = do
  let fileName = "data/input.csv"
  input <- readFile fileName
  let csv = parseCSV fileName input
  either handleError doWork csv

handleError csv = putStrLn "not a CSV"
doWork csv = print $ filter (\row -> last row == "1") $ rows csv

rows :: [[a]] -> [[a]]
rows csv = filter (\x -> length x == length (head csv)) $ tail csv

main

[["Lois Williams","lois.williams50@example.com","2015-02-01T10:23:45.964Z","1"],["Russell Martin","russell.martin@example.com","2015-02-01T10:23:45.964Z","1"],["Bruce Shaw","bruce.shaw@example.com","2015-02-01T10:23:45.964Z","1"]]

In [7]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Aws
import qualified Aws.S3 as S3
import           Data.Conduit (($$+-))
import           Data.Conduit.Binary (sinkFile)
import           Network.HTTP.Conduit (withManager, responseBody)

main :: IO ()
main = do
  cfg <- Aws.baseConfiguration
  let s3cfg = Aws.defServiceConfig :: S3.S3Configuration Aws.NormalQuery

  withManager $ \mgr -> do
    S3.GetObjectResponse { S3.gorResponse = rsp } <-
      Aws.pureAws cfg s3cfg mgr $
        S3.getObject "haskell-aws" "cloud-remote.pdf"

    responseBody rsp $$+- sinkFile "cloud-remote.pdf"
    
    
main